In [8]:
import streamlit as st
import os
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta
from dotenv import load_dotenv
from supabase import create_client, Client
from streamlit_autorefresh import st_autorefresh
from zoneinfo import ZoneInfo  # [ALTERADO] usar ZoneInfo para TZ

In [9]:
# Carrega as variáveis do arquivo .env
load_dotenv()
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

# Inicializa cliente
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# [NOVO] Constante de fuso para São Paulo
TZ_SP = ZoneInfo("America/Sao_Paulo")

# [NOVO] Cache leve para paginação Supabase (evita re-hit constante ao rolar a página)
#@st.cache_data(show_spinner=True, ttl=300)
def ler_dados_supabase(tabela: str, pagina_tamanho: int = 1000) -> pd.DataFrame:
    offset = 0
    dados_completos = []

    while True:
        resposta = (
            supabase
            .table(tabela)
            .select("*")
            .range(offset, offset + pagina_tamanho - 1)
            .execute()
        )
        dados = resposta.data
        if not dados:
            break
        dados_completos.extend(dados)
        offset += pagina_tamanho

    df = pd.DataFrame(dados_completos)

    # [ALTERADO] Normalização da coluna DataHoraReal: ISO8601 -> tz-aware UTC -> tz São Paulo -> tz-naive (preserva comportamento)
    # Motivo: manter mesma lógica de .dt.date e gráficos, mas com horário convertido para o "relógio" de São Paulo.
    if "DataHoraReal" in df.columns and not df.empty:
        df["DataHoraReal"] = (
            pd.to_datetime(df["DataHoraReal"], utc=True, errors="coerce")
              .dt.tz_convert(TZ_SP)
              .dt.tz_localize(None)  # remove o tz, horário já está em SP
        )

    return df

# === Carrega dados e aplica filtro fixo para fontes sólidas ===
df = ler_dados_supabase("resultados_analiticos")
fontes_s = ["LIX_Au_S", "TQ2_Au_S", "TQ5_Au_S", "TQ6_Au_S", "REJ_Au_S"]
df = df[df["Fonte"].isin(fontes_s)]

In [10]:
df

,id,Fonte,DataHoraReal,Valor,MediaMovel_6
16,127773,LIX_Au_S,2025-11-10 23:59:00,0.734,0.720833
18,127775,TQ5_Au_S,2025-11-10 23:59:00,0.186,0.146333
21,127778,REJ_Au_S,2025-11-10 23:59:00,0.126,0.109667
30,127787,LIX_Au_S,2025-11-10 16:00:00,0.564,0.724000
37,127794,TQ6_Au_S,2025-11-10 16:00:00,0.091,0.109167
...,...,...,...,...,...
31961,159718,LIX_Au_S,2023-06-13 12:00:00,0.880,0.777333
31962,159719,REJ_Au_S,2023-06-12 23:59:00,0.054,0.048500
31963,159720,LIX_Au_S,2023-06-12 23:59:00,0.778,0.726000
31964,159721,REJ_Au_S,2023-06-12 12:00:00,0.043,0.043000


In [11]:
if df.empty:
    st.warning("Nenhum dado disponível para as fontes sólidas.")
    st.stop()

# [NOVO] Garantir ordenação temporal antes de cálculos
df = df.sort_values(["Fonte", "DataHoraReal"], kind="stable")

# === Datas ===
# [ALTERADO] datas já estão no horário local (SP) e tz-naive
if "DataHoraReal" not in df.columns or df["DataHoraReal"].dropna().empty:
    st.error("⚠️ Nenhuma data válida encontrada em DataHoraReal.")
    st.stop()

data_max = df["DataHoraReal"].max()
data_min_total = df["DataHoraReal"].min()

if pd.isna(data_max) or pd.isna(data_min_total):
    st.error("⚠️ Datas inválidas detectadas (NaT). Verifique o conteúdo de DataHoraReal.")
    st.stop()

data_min_default = data_max - pd.Timedelta(days=30)

In [12]:
print(data_max)
print(data_min_total)

2025-11-10 23:59:00
2023-06-12 12:00:00


In [13]:
df

,id,Fonte,DataHoraReal,Valor,MediaMovel_6
31965,159722,LIX_Au_S,2023-06-12 12:00:00,0.674,0.674000
31963,159720,LIX_Au_S,2023-06-12 23:59:00,0.778,0.726000
31961,159718,LIX_Au_S,2023-06-13 12:00:00,0.880,0.777333
31958,159715,LIX_Au_S,2023-06-14 12:00:00,0.656,0.747000
31957,159714,LIX_Au_S,2023-06-14 23:59:00,0.842,0.766000
...,...,...,...,...,...
101,127858,TQ6_Au_S,2025-11-09 08:00:00,0.127,0.116667
84,127841,TQ6_Au_S,2025-11-09 16:00:00,0.149,0.124000
63,127820,TQ6_Au_S,2025-11-09 23:59:00,0.094,0.118667
56,127813,TQ6_Au_S,2025-11-10 08:00:00,0.096,0.111333
